In [65]:
import notebook_common
import jaconv
import os
from importlib import reload

reload(notebook_common)
from notebook_common import (
    # ------------------function------------------
    # read_excel,
    to_csv_sig,
    # ------------------2400 word------------------
    # r_word_2400_xlsx,
    # w_word_2400_csv,
    # w_word_2400_with_guide_csv,
    w_word_2400_without_guide_csv,
    # ------------------词性字典表------------------
    r_pos_dict_csv,
    # ------------------书面语相关------------------
    # 解析书面语正则
    # pattern_table_written,
    # pattern_td_written,
    # r_written_freq_nlt_with_enter_txt,
    # w_written_freq_nlt_with_enter_csv,
    w_written_freq_nlt_sumed_csv,
    # ------------------口语相关------------------
    # 解析口头语正则
    w_spoken_freq_nlb_sumed_csv,
    # ------------------合并书面语和口语词频------------------
    w_merged_freq_sumed_csv,
    w_raw_merged_freq_sumed_csv,
    w_word_raw_merged_freq_sumed_csv,
    # ------------------发音词频表------------------
    r_pron_freq_csv,
    # ------------------处理词义相关------------------
    # ------------------公共python库------------------
    pd,
    np,
    p,
    l,
)

p("Import common module")

Import common module


In [66]:
df_meaning_2400_normal = pd.read_csv(
    "d:/tmp/normal.csv",
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    # index_col=["spell", "pron"],
)

# 词性字典表
df_pos_dict = pd.read_csv(
    r_pos_dict_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    # keep_default_na=False,
    # index_col=["word", "reading"]
)


# 发音词频表
df_pron_freq = pd.read_csv(
    r_pron_freq_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    # keep_default_na=False,
    index_col=["spell", "pron"]
)


# df_pron_freq = df_pron_freq[~df_pron_freq['spell'].isin(['％', '○○'])]


df_meaning_2400_normal[0:5]

,spell,pron,pron_per,pos,pos_per,meaning
0,ああ,ああ,100,感叹词,100,[(感叹/表示惊讶或感叹:100%)]
1,愛,あい,100,名词,100,"[(爱/深厚的感情:70%),(慈爱/关怀:30%)]"
2,合図,あいず,100,名词,80,"[(信号/示意:70%),(暗号/约定信号:30%)]"
3,合図,あいず,100,动3,20,[(示意/用动作或声音提示:100%)]
4,アイスクリーム,アイスクリーム,100,名词,100,[(冰淇淋/冷冻甜品:100%)]


In [67]:
df_pron_freq[0:3]

,,pron_id,wordId,roman,pron_index,pron_percent,voice,pron_voice,word_fused_freq,fused_freq,freq_for_order,word_voice,sum_mix_pron_freq,mix_pron_freq,spoken_pron_freq,written_pron_freq
spell,pron,,,,,,,,,,,,,,,
の,の,10000001.0,100000.0,no,1.0,1.0,skent/の.mp3|poyotan/の.mp3|otiose/の.mp3|akiconf...,skent/の.mp3|poyotan/の.mp3|otiose/の.mp3|akiconf...,3273064.0,3273064.0,72395271.0,skent/の.mp3|poyotan/の.mp3|otiose/の.mp3|akiconf...,43219580.0,43219580.0,57221425.0,686025.0
いる,いる,10000101.0,100001.0,iru,1.0,1.0,strawberrybrown/いる.mp3|yasuo/いる.mp3,strawberrybrown/いる.mp3|yasuo/いる.mp3,1622707.0,1622707.0,35891854.0,strawberrybrown/いる.mp3|yasuo/いる.mp3,16052825.0,16052825.0,12277963.0,1067608.0
に,に,10000201.0,100002.0,NaN,1.0,1.0,akiconfig/に.mp3|otiose/に.mp3|strawberrybrown/に...,akiconfig/に.mp3|otiose/に.mp3|strawberrybrown/に...,1595358.0,1595358.0,35286930.0,akiconfig/に.mp3|otiose/に.mp3|strawberrybrown/に...,23536382.0,23536382.0,35286930.0,0.0


In [68]:
df_pos_dict[0:5]

,pos_id,parent_id,pos,verb_zita,verb_is_sup,verb_is_aux,is_optional
0,10,0,名词,NaN,NaN,NaN,1
1,11,0,代词,NaN,NaN,NaN,1
2,12,0,接头词,NaN,NaN,NaN,1
3,13,0,接尾词,NaN,NaN,NaN,1
4,14,0,感叹词,NaN,NaN,NaN,1


In [69]:
# 关联词性ID
df_meaning_2400_normal["pos_id"] = (
    df_meaning_2400_normal["pos"]
    .map(df_pos_dict.set_index("pos")["pos_id"])
    .astype("Int64")
)
# df_meaning_2400_normal["pos_id"] = df_meaning_2400_normal["pos_id"].astype(int)

# 将发音都改为平假名
df_meaning_2400_normal["pron"] = df_meaning_2400_normal["pron"].apply(jaconv.kata2hira)
# to_csv_sig(df_meaning_2400_problem, "d:/tmp/problem_pron_transformed.csv")
# l("df_meaning_2400_problem")


df_meaning_2400_normal.set_index(["spell", "pron"], inplace=True)
df_merged = df_meaning_2400_normal.merge(
    df_pron_freq[["pron_id", "wordId", "mix_pron_freq"]],
    left_index=True,
    right_index=True,
    how="left",
    indicator=True,
)
df_merged = df_merged.reset_index()
to_csv_sig(df_merged, "d:/tmp/normal_merged.csv")
l("df_merged")
# df_merged

----------------------------------------df_merged----------------------------------------
